## Creating Paired Dataset

This is used for creating the paired dataset given a source image, instruction, target image for training the diffusion model.

In [1]:
import pandas as pd
import os
import random

<h4>Manuually sort the txt file

In [2]:
file_name = 'monet.txt cezanne.txt ukiyoe.txt vangogh.txt'
file_name = file_name.split(' ')
file_name

['monet.txt', 'cezanne.txt', 'ukiyoe.txt', 'vangogh.txt']

In [3]:
# remove extra newline in vangogh.txt
with open('vangogh.txt', 'r') as f:
    lines = f.readlines()
with open('vangogh.txt', 'w') as f:
    for line in lines:
        if line != '\n':
            f.write(line)

<h4>Get the entire instruction file

In [3]:
instruct = []
for i in range(len(file_name)):
    file = file_name[i]
    f = open(file, 'r')
    prompt = f.read()
    f.close()
    
    prompt = prompt.split('\n')[:-1]

    if prompt[0][0] == '"':
        prompt = [p[1:-1]for p in prompt]
    else:
        prompt = [p for p in prompt]
    
    selected_items = random.choices(prompt, k=2000)
    instruct.extend(selected_items)

In [5]:
len(instruct)

8000

In [6]:
with open("instruction.txt", 'w') as file:
    for sentence in instruct:
        file.write(sentence + '\n')

<h4> Paired in DataFrame

In [3]:
paired_df = pd.DataFrame()
paired_df['source_img'] = None
paired_df['instruction'] = None
paired_df['target_img'] = None

def num(x):
    return int(x.split(".")[0])
    
source = os.listdir("custom_datasets_3/source")
#source = sorted(source)[1:]
source = sorted(source, key = num)
target = os.listdir("custom_datasets_3/target")

In [4]:
f = open('instruction.txt', 'r')
prompt = f.read().split('\n')[:-1]
f.close()
len(prompt)

8000

In [5]:
modified_prompt = []
for ele in prompt:
    if ele.endswith('"'):
        modified_prompt.append(ele[1:-1])
    else:
        modified_prompt.append(ele)

In [6]:
len(modified_prompt)

8000

In [7]:
paired_df['source_img'] = source
paired_df['instruction'] = modified_prompt

In [8]:
for i, row in enumerate(paired_df.itertuples()):
    target_filename = row.source_img.split('.')[0] + '.png'
    if target_filename in target:
        paired_df.at[i, 'target_img'] = target_filename

In [9]:
paired_df['source_img'] = paired_df['source_img'].apply(lambda x: os.path.join('custom_datasets_3/source', x))
paired_df['target_img'] = paired_df['target_img'].apply(lambda x: os.path.join('custom_datasets_3/target', x))

In [10]:
paired_df.head()

,source_img,instruction,target_img
0,custom_datasets_3/source/1.png,Drip Monet's artistic splendor into this image.,custom_datasets_3/target/1.png
1,custom_datasets_3/source/2.png,Reshape the image in Monet's artistic flair.,custom_datasets_3/target/2.png
2,custom_datasets_3/source/3.png,Give image a Monet-style rendition.,custom_datasets_3/target/3.png
3,custom_datasets_3/source/4.png,Imbue the image in the hues of Monet.,custom_datasets_3/target/4.png
4,custom_datasets_3/source/5.png,Replace the style of the image with that of Mo...,custom_datasets_3/target/5.png


In [11]:
# split dataframe to half
paired_df1 = paired_df[:4000].copy()
paired_df2 = paired_df[4000:].copy()

In [12]:
from PIL import Image
def convert_image(path):
    try:
        img = Image.open(path)
        return img
    except Exception as e:
        print(f"Error loading image {path}")
        print(e)
        return None

In [13]:
paired_df1['source_img'] = paired_df1['source_img'].apply(lambda x: convert_image(x))
paired_df1['target_img'] = paired_df1['target_img'].apply(lambda x: convert_image(x))

In [14]:
paired_df1 = paired_df1.dropna()

In [15]:
paired_df1.head()

,source_img,instruction,target_img
0,<PIL.PngImagePlugin.PngImageFile image mode=RG...,Drip Monet's artistic splendor into this image.,<PIL.PngImagePlugin.PngImageFile image mode=RG...
1,<PIL.PngImagePlugin.PngImageFile image mode=RG...,Reshape the image in Monet's artistic flair.,<PIL.PngImagePlugin.PngImageFile image mode=RG...
2,<PIL.PngImagePlugin.PngImageFile image mode=RG...,Give image a Monet-style rendition.,<PIL.PngImagePlugin.PngImageFile image mode=RG...
3,<PIL.PngImagePlugin.PngImageFile image mode=RG...,Imbue the image in the hues of Monet.,<PIL.PngImagePlugin.PngImageFile image mode=RG...
4,<PIL.PngImagePlugin.PngImageFile image mode=RG...,Replace the style of the image with that of Mo...,<PIL.PngImagePlugin.PngImageFile image mode=RG...


In [16]:
len(paired_df1)

4000

In [17]:
from datasets import Dataset, Features, Image, Value

In [18]:
df_to_dict = paired_df1.to_dict('list')
features = Features({
    'source_img': Image(decode=True),
    'instruction': Value(dtype='string'),
    'target_img': Image(decode=True)
})

In [19]:
dataset = Dataset.from_dict(df_to_dict, features=features)

In [38]:
from huggingface_hub import login
login()

In [20]:
dataset.push_to_hub('tianyi0216/BigDS_Cycle', private=False)

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/370 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/tianyi0216/BigDS_Cycle/commit/64c12b04c5a2655b2e1c12d74dbbb1ac9a9d9f1f', commit_message='Upload dataset', commit_description='', oid='64c12b04c5a2655b2e1c12d74dbbb1ac9a9d9f1f', pr_url=None, pr_revision=None, pr_num=None)